In [1]:
import sqlite3
import numpy as np
from matplotlib import pyplot as plt

#수집시간(5초)
COLLECTION_TIME = 5000

In [ ]:
def draw_plot(X, Y, title):
    for i in range(len(X)):
        plt.plot(X[i], Y[i])
    
    plt.xlabel('sec')
    plt.ylabel('value')
    plt.title(title)
    plt.show()

In [132]:
fn_move_db = 'BlackBox/BlackBox_move.db'
move_conn = sqlite3.connect(fn_move_db)
cur = move_conn.cursor()

In [2]:
fn_stay_db = 'BlackBox/BlackBox_stay.db'
stay_conn = sqlite3.connect(fn_stay_db)
cur = stay_conn.cursor()

## Based on AP Similarity

In [3]:
def get_fingerprint_list(rows, key = lambda x : (x['rss'], x['bssid']), reverse=True):
    fp_dict = dict()
    prev_time = -1e8
    
    for row in rows:
        element = {
            'timeMicros': row[1],
            'ssid': row[2],
            'bssid': row[3],
            'freq': row[4],
            'rss': row[5]
        }
        wifi_time = row[6]
        
        try:
            fp_dict[wifi_time].append(element)
        except KeyError:
            fp_dict[wifi_time] = [element]
    
    fp_list = [{'logTime': t, 'data': sorted(fp_dict[t], key=key, reverse=reverse)} for t in sorted(fp_dict)]
    return fp_list

In [4]:
cur.execute("select * from Rss")
rows = cur.fetchall()

fp_list = get_fingerprint_list(rows)

In [7]:
top_n = 10
move_count = 0

# Need to consider when len(fp_list) == 1
for prev_fp, cur_fp in zip(fp_list, fp_list[1:]):
    prev_bssids = [row['bssid'] for row in prev_fp['data'][:top_n]]
    cur_bssids = [row['bssid'] for row in cur_fp['data'][:top_n]]
    
    intersection = list(set(prev_bssids) & set(cur_bssids))
    
    #print([(row['ssid'], row['bssid'], row['rss']) for row in prev_fp['data'][:top_n]])
    #print([(row['ssid'], row['bssid'], row['rss']) for row in cur_fp['data'][:top_n]])
    
    if len(intersection) < top_n * 0.7:
        move_count += 1
    
    print(intersection)
    print()


if move_count > len(fp_list) * 0.5:
    print('moved')
else:
    print('stayed')


['18:31:bf:46:ff:d8', '90:9f:33:73:21:d6', '9c:d6:43:c8:11:80', '70:5d:cc:92:31:08', '60:38:e0:bb:ee:01', '70:5d:cc:93:31:08', '00:08:9f:bf:5e:44']

['18:31:bf:46:ff:d8', '62:38:e0:bb:ee:01', '90:9f:33:73:21:d6', '9c:d6:43:c8:11:80', 'c4:e9:84:f8:c7:73', '70:5d:cc:92:31:08', '60:38:e0:bb:ee:01', '70:5d:cc:93:31:08', '00:08:9f:bf:5e:44']

['18:31:bf:46:ff:d8', '62:38:e0:bb:ee:01', '90:9f:33:73:21:d6', 'c4:e9:84:f8:c7:73', '70:5d:cc:92:31:08', '60:38:e0:bb:ee:01', '70:5d:cc:93:31:08', '00:08:9f:bf:5e:44']

['18:31:bf:46:ff:d8', '62:38:e0:bb:ee:01', '90:9f:33:73:21:d6', '9c:30:5b:e7:70:3a', 'c4:e9:84:f8:c7:73', '70:5d:cc:92:31:08', '60:38:e0:bb:ee:01', '70:5d:cc:93:31:08', '00:08:9f:bf:5e:44']

['18:31:bf:46:ff:d8', '62:38:e0:bb:ee:01', '72:5d:cc:48:5b:28', '9c:30:5b:e7:70:3a', 'c4:e9:84:f8:c7:73', '70:5d:cc:92:31:08', '60:38:e0:bb:ee:01', '70:5d:cc:93:31:08', '00:08:9f:bf:5e:44']

['18:31:bf:46:ff:d8', '62:38:e0:bb:ee:01', '72:5d:cc:48:5b:28', 'c4:e9:84:f8:c7:73', '70:5d:cc:92:31:08', '6

## Based on Accerlometer Data

In [6]:
#Moving or Stay
MIN_LENGTH = 50

acc_list = list()
cur.execute("select * from Acc")
rows = cur.fetchall()

prev_time = -1e8
for row in rows:
    #print(row)
    element = {
        'timeMillis': row[1],
        'timeNanos': row[2],
        'x': row[3],
        'y': row[4],
        'z': row[5]
    }
    
    if (element['timeMillis'] - prev_time) < COLLECTION_TIME:
        acc_list[-1].append(element)
    else:
        acc_list.append([element])
        prev_time = element['timeMillis']


#수집된 데이터가 너무 적은 경우 제외
idx = 0
while idx < len(acc_list):
    if len(acc_list[idx]) < MIN_LENGTH:
        acc_list.pop(idx)
    else:
        idx += 1
        
print('len(acc_list):', len(acc_list))

len(acc_list): 10


In [7]:
count = 0
VAR_THRESHOLD = 10
COUNT_THRESHOLD = 0.5

for i in range(len(acc_list)):
    x_axis = np.array([ele['x'] for ele in acc_list[i]])
    y_axis = np.array([ele['y'] for ele in acc_list[i]])
    z_axis = np.array([ele['z'] for ele in acc_list[i]])
    
    if max(x_axis.var(), y_axis.var(), z_axis.var()) > VAR_THRESHOLD:
#         x_line = np.linspace(0, 5, len(acc_list[i]))
#         draw_plot([x_line, x_line, x_line], [x_axis, y_axis, z_axis], str(i) + '-th')
        count += 1

if count / len(acc_list) >= COUNT_THRESHOLD:
    print('moved')
else:
    print('stayed')

print('len(acc_list):', len(acc_list))
print('moved_count:', count)
print('stayed_count:', len(acc_list) - count)

stayed
len(acc_list): 10
moved_count: 2
stayed_count: 8


In [137]:
move_conn.close()
stay_conn.close()